# Glacier mass balance ML models

# Part I: Producing a dataset

First, we import the dependencies and we configure OGGM.

In [122]:
import xarray as xr
import numpy as np
import rioxarray
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import cfg, utils, workflow, tasks, graphics
from oggm import entity_task, global_tasks
from oggm.utils import compile_climate_input 
from oggm.core import gis
from oggm.utils import DEM_SOURCES
from pathlib import Path
import os
import logging

pd.set_option('display.max_columns',None)

cfg.initialize(logging_level='WARNING')
cfg.PARAMS['border'] = 10
cfg.PARAMS['use_multiprocessing'] = True 
# Module logger
log = logging.getLogger('.'.join(__name__.split('.')[:-1]))

2022-06-24 14:28:45: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2022-06-24 14:28:45: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2022-06-24 14:28:45: oggm.cfg: Multiprocessing: using all available processors (N=64)
2022-06-24 14:28:45: oggm.cfg: PARAMS['border'] changed from `40` to `10`.
2022-06-24 14:28:45: oggm.cfg: Multiprocessing switched ON after user settings.


Choose your OGGM path where you want to store all the data.

In [4]:
parent_path = os.path.dirname(Path().resolve())
workspace_path = os.path.join(parent_path, 'OGGM_data_Finse')
if not os.path.exists(workspace_path):
    os.mkdir(workspace_path)
else:
    cfg.PATHS['working_dir'] = workspace_path

Download all data from glaciers in Scandinavia (RGI region '08').

In [5]:
rgi_region = '08'
rgi_version = '6'
rgi_dir = utils.get_rgi_dir(version=rgi_version)
path = utils.get_rgi_region_file(region=rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)
gdirs = workflow.init_glacier_directories(rgidf, from_prepro_level=3, prepro_border=10)

2022-06-24 12:56:22: oggm.workflow: init_glacier_directories from prepro level 3 on 3417 glaciers.
2022-06-24 12:56:22: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 3417 glaciers


## Get geodetic glacier mass balance data

We get the geodetic MB for all glaciers in Scandinavia from Hugonnet et al. (2021)

In [ ]:
mbdf = utils.get_geodetic_mb_dataframe()

In [ ]:
mbdf.drop(columns=['area', 'reg', 'is_cor'])

In [ ]:
mbdf.index.names = ['RGI_ID']

## Get glacier topographical data

Now we get the topographical data for all glacier to be used in the training. 

In [ ]:
# TODO: COMPUTE AVERAGE ICE THICKNESS PER GLACIER

In [ ]:
gdir = gdirs[100]

In [ ]:
glshp = gdir.read_shapefile('outlines')

In [ ]:
glshp.O1Region.values[0]

In [ ]:
dem_path = gdir.get_filepath('dem')

In [ ]:
da = rioxarray.open_rasterio(dem_path)
f, ax = plt.subplots()
da.plot(cmap='terrain', ax=ax);
# Add the outlines
gdir.read_shapefile('outlines').plot(ax=ax, color='none', edgecolor='black');

In [43]:
@entity_task(log)
def get_topo_predictors(gdir):
    """Mandatory docstring
    """
        
    training_data = {'zmed': 0.0,
                 'zmax': 0.0,
                 'zmin': 0.0,
                 'area': 0.0,
                 'slope': 0.0,
                 'lat': 0.0,
                 'icecap': 0.0,
                 'ID': ""
        }
    
    gl_shp = gdir.read_shapefile('outlines')
    
    training_data['zmed'] = float(gl_shp.Zmed.values[0])
    training_data['zmax'] = float(gl_shp.Zmax.values[0])
    training_data['zmin'] = float(gl_shp.Zmin.values[0])
    training_data['area'] = gdir.rgi_area_km2
    training_data['lat'] = gdir.cenlat
    training_data['icecap'] = int(gdir.is_icecap)
    training_data['slope'] = float(gl_shp.Slope.values[0])
    training_data['ID'] = gdir.rgi_id
        
    return training_data

We parallelize this using the function as an entity task in OGGM

In [46]:
topo_dicts = workflow.execute_entity_task(get_topo_predictors, gdirs)

2022-06-24 13:50:53: oggm.workflow: Execute entity tasks [get_topo_predictors] on 3417 glaciers


In [47]:
topo_df = pd.DataFrame(topo_dicts)
topo_df.index = topo_df.ID
topo_df.index.name = 'RGI_ID'
topo_df.drop(columns='ID')

,zmed,zmax,zmin,area,slope,lat,icecap
RGI_ID,,,,,,,
RGI60-08.00001,242.0,250.0,235.0,0.030,5.8,67.930,0
RGI60-08.00002,239.0,246.0,228.0,0.030,5.7,67.870,0
RGI60-08.00003,726.0,749.0,701.0,0.020,18.8,67.820,0
RGI60-08.00004,696.0,725.0,671.0,0.020,20.1,67.830,0
RGI60-08.00005,1268.0,1807.0,926.0,21.469,8.5,67.139,0
...,...,...,...,...,...,...,...
RGI60-08.03413,980.0,1182.0,887.0,0.098,38.6,68.164,0
RGI60-08.03414,1377.0,1452.0,1181.0,0.389,17.6,66.590,0
RGI60-08.03415,1314.0,1330.0,1289.0,0.056,10.7,66.729,0


In [48]:
topo_df.to_csv('topo_df.csv')

To avoid computing all topographical predictors each time, just load the previously stored file.

In [44]:
topo_df = pd.read_csv('topo_df.csv')

## Get glacier climate data

Now we get the climate data from CRU for each glacier. 

In [ ]:
global_tasks.compile_climate_input(gdirs)

In [80]:
climate_ds = xr.open_dataset('climate_input.nc')

In [84]:
climate_ds

<xarray.Dataset>
Dimensions:         (time: 1416, rgi_id: 3417)
Coordinates:
  * time            (time) float64 1.902e+03 1.902e+03 ... 2.02e+03 2.02e+03
  * rgi_id          (rgi_id) object 'RGI60-08.00001' ... 'RGI60-08.03417'
    hydro_year      (time) int64 ...
    hydro_month     (time) int64 ...
    calendar_year   (time) int64 ...
    calendar_month  (time) int64 ...
Data variables:
    temp            (time, rgi_id) float32 ...
    prcp            (time, rgi_id) float32 ...
    ref_hgt         (rgi_id) float64 ...
    ref_pix_lon     (rgi_id) float64 ...
    ref_pix_lat     (rgi_id) float64 ...
    zmed            (rgi_id) float64 242.0 239.0 726.0 ... 1.581e+03 1.186e+03
Attributes:
    description:    OGGM model output
    oggm_version:   1.5.3
    calendar:       365-day no leap
    creation_date:  2022-06-24 13:19:30

In [88]:
climate_ds.temp.data = climate_ds.temp.data + 6.0/1000.0*(climate_ds.zmed.data - climate_ds.ref_hgt.data) # Super rough temperature lapse rate  

In [178]:
climate_ds.temp.where(climate_ds.temp > 0.0).groupby('hydro_year').sum()

<xarray.DataArray 'temp' (hydro_year: 118, rgi_id: 3417)>
array([[26.1888899 , 30.69222162, 22.27333452, ..., 21.15333549,
        47.44399936, 37.56999868],
       [30.77777855, 35.38148187, 26.79666834, ..., 23.94333498,
        49.78377692, 38.50333244],
       [29.17777781, 33.74814783, 26.05600025, ..., 23.43533476,
        48.14755413, 37.81999773],
       ...,
       [34.98888914, 40.15155528, 31.07333447, ..., 31.66133547,
        61.75644371, 50.00333125],
       [46.21111195, 51.66266701, 42.19666793, ..., 38.56133604,
        68.58866533, 57.00444382],
       [35.64444439, 40.75155625, 31.59666814, ..., 32.76133528,
        68.15533277, 54.64333151]])
Coordinates:
  * rgi_id      (rgi_id) object 'RGI60-08.00001' ... 'RGI60-08.03417'
  * hydro_year  (hydro_year) int64 1902 1903 1904 1905 ... 2016 2017 2018 2019

In [134]:
def get_climate_predictors(climate_ds, period=pd.date_range(start="2000-01-01",end="2020-01-01")):
    """Mandatory docstring
    """
    climate_dict = {'PDD':[],
                    'snowfall':[],
                    'rainfall':[],
                    'years':[]
    }
     
    climate_ds = climate_ds.sel(time=period, method='nearest') # we trim the data for the desired period

    # Temperature
    climate_ds.temp.data = climate_ds.temp.data + 6.0/1000.0*(climate_ds.zmed.data - climate_ds.ref_hgt.data) # Super rough temperature lapse rate  
    climate_dict['PDD'] = climate_ds.temp.where(climate_ds.temp > 0.0).groupby('hydro_year').sum().data
    
    # Snowfall
    climate_dict['snowfall'] = climate_ds.prcp.where(climate_ds.temp <= 0.0).groupby('hydro_year').sum().data
    # Rainfall
    climate_dict['rainfall'] = climate_ds.prcp.where(climate_ds.temp > 0.0).groupby('hydro_year').sum().data
    #Years
    climate_dict['years'] = np.unique(climate_ds.hydro_year)
    
    print(climate_dict)
    
    # Convert dict to dataframe
    #climate_df = pd.DataFrame(climate_dict)
    
    return climate_dict

In [ ]:
# TODO: format climate data in yearly cumulative data per glacier.
# Simulations for 20 years (2000-2020) will have 20 predictors (one per year). Simulations for 10 years (2000-2010 or 2010-2020), 
# will have 10 predictors filled and 10 predictors at 0. And same with simulations for 5 years (if the SNR is reasonable). 

In [135]:
climate_df = get_climate_predictors(climate_ds)

{'PDD': array([[ 40152.15654755,  49809.06686549,  40460.63580365, ...,
         45365.39112167, 100394.18347004,  72840.82529534]]), 'snowfall': array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'rainfall': array([[ 435159.38,  443665.97,  439780.97, ...,  744597.  , 1023729.8 ,
        1000944.56]], dtype=float32), 'years': array([2019])}


In [161]:
climate_df['PDD'][0][0]

40152.15654755483